# Stroke predicition

In this project, we use the dataset to predict who is likely to have a stroke by using the information like age, gender, bmi, smoking status and the other diseases.

The information of each column is provided below. 

1) id: unique identifier
2) gender: "Male", "Female" or "Other"
3) age: age of the patient
4) hypertension: 0 if the patient doesn't have hypertension, 1 if the patient has hypertension
5) heart_disease: 0 if the patient doesn't have any heart diseases, 1 if the patient has a heart disease
6) ever_married: "No" or "Yes"
7) work_type: "children", "Govt_jov", "Never_worked", "Private" or "Self-employed"
8) Residence_type: "Rural" or "Urban"
9) avg_glucose_level: average glucose level in blood
10) bmi: body mass index
11) smoking_status: "formerly smoked", "never smoked", "smokes" or "Unknown"
12) stroke: 1 if the patient had a stroke or 0 if not


Note: "Unknown" in smoking_status means that the information is unavailable for this patient

# Exploratory Data Analysis

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

In [ ]:
df=pd.read_csv('healthcare-dataset-stroke-data.csv')

We now first discovering by using head and shape.

In [ ]:
df.head(10)

In [ ]:
df.shape

In this dataset, there are 12 columns and 5110 rows. Next, we need some descriptive statistics on this dataset.

In [ ]:
df.describe(include='all')

In the 'gender' column, there are 3 unique values, Male, Female, and Other. The average age of these observation is around 43 years old. Most of the patient are never smoke before. The average glucose level and bmi among all of the patients are 106.147677 and 28.893237, respectively.

In [ ]:
df.info()

We can see that in the 'bmi' column, there are some value missing. In the further regression, we will delete the row with these value missing.

In [ ]:
df.duplicated().value_counts()

There are no duplicate in this dataset.

In [ ]:
df['stroke'].value_counts(normalize=True)*100

We can see that only 4% of the dataset are the people who have got stroke, which is too imbalanced.

In [ ]:
df['gender'].value_counts()

We can see that 2994 patients are female, and 2115 are males, but notice that there are one other gender. Let's check what is it.

In [ ]:
df[df['gender'] == 'Other']

Since this row of this dataset does not have any value that is seriously have negative effect of this data, we can leave this data by do not delete it.\ Now, we plot the histogram on the age distribution to see how is the dataset observed.

In [ ]:
plt.figure(figsize=(10,5))
sns.histplot(df['age'])
plt.title('Age histogram');

We observe that this data collect all patients in all range of ages. Next, we plot a boxplot to see if there is any outliers here.

In [ ]:
plt.figure(figsize=(10,5))
plt.title('Average glucose level boxplot')
sns.boxplot(x=df['avg_glucose_level']);

And check outliers for bmi.

In [ ]:
plt.figure(figsize=(10,5))
plt.title('BMI boxplot')
sns.boxplot(x=df['bmi']);

There are some values that are maybe too high, so we observe these values whether it is possible to have these highly bmi.

In [ ]:
high_bmi = df[df['bmi'] >= 65]
high_bmi

We have a question on these outliers. We need to think that these outliers may be affect when we do a regression model. We will consider it later.


Below are the three plots of the relationship between average glucose level, bmi, and stroke.

In [ ]:
plt.figure(figsize=(12,7))
ax = sns.histplot(data=df, x='avg_glucose_level', hue='stroke', multiple='stack')
plt.title('Average glucose level by stroke histogram');

In [ ]:
plt.figure(figsize=(12,7))
ax = sns.histplot(data=df, x='bmi', hue='stroke', multiple='stack')
plt.title('BMI by stroke histogram');

We can see that both plots are in the bell-shape, that means the dataset we observed have a normal distribution.


Belows are the pie charts show the percentage of categorical variables.

In [ ]:
plt.pie(df['ever_married'].value_counts(), labels=df['ever_married'].unique(), autopct='%1.1f%%');
plt.title('Percentage of married (both still and former) people');

In [ ]:
plt.pie(df['work_type'].value_counts(), labels=df['work_type'].unique(), autopct='%1.1f%%');
plt.title('Percentage of types of work');

In [ ]:
plt.pie(df['Residence_type'].value_counts(), labels=df['Residence_type'].unique(), autopct='%1.1f%%');
plt.title('Percentage of residence types');

In [ ]:
plt.pie(df['smoking_status'].value_counts(), labels=df['smoking_status'].unique(), autopct='%1.1f%%');
plt.title('Percentage of smoking status');

In [ ]:
plt.figure(figsize=(12,7))
ax = sns.histplot(data=df, x='avg_glucose_level', hue='gender', multiple='stack')
plt.title('The distribution of average glucose level on gender');

Both genders have the same normal distribution of average glucose level.

In [ ]:
plt.figure(figsize=(12,7))
ax = sns.histplot(data=df, x='bmi', hue='smoking_status', multiple='layer', kde=True)
plt.title('The distribution of bmi base on smoking status');

There is a difference on the distribution of bmi between unknown group and the others, and there is a bit difference on the left three group with each other.

In [ ]:
plt.figure(figsize=(12,7))
ax = sns.histplot(data=df, x='bmi', hue='hypertension', multiple='layer', kde=True)
plt.title('The distribution of bmi base on hypertension category');
plt.axvline(x=df[df['hypertension'] == 1].bmi.mean(), color='red', ls='--', lw=2.5);
plt.axvline(x=df[df['hypertension'] == 0].bmi.mean(), color='blue', ls='--',lw=2.5);

A little bit difference of mean between both group of who have got hypertension has shown.


The below scatter plot shows the relationship between average glucose level and bmi. We can see that most of the data are clustered around the bottom left of the plot.

In [ ]:
plt.figure(figsize=(12,7))
ax = sns.scatterplot(data=df, x='avg_glucose_level', y='bmi', marker="+")
plt.title('Scatterplot of average glucose level and bmi');

Three assumptions that may be affect how stroke will be occur are hypertension, heart disease and smoking status. We will observe the percentage of stroke categorized by these causes.

In [ ]:
df.groupby(['hypertension', 'heart_disease'])[['stroke']].sum()

It is interesting that although the patients have no hypertension and heart disease, stroke still can be occurred with highly values.


Now, consider on the people who have got stroke. We plot to see the distribution of age among all of these group of stroke.

In [ ]:
strokep = df[df['stroke'] == 1]
strokep.head()

In [ ]:
strokep['stroke'].count()

There are 249 people that have got stroke.

In [ ]:
strokep.describe()

Around 27% of people who have stroke and 19% who have stroke have got hypertension and heart disease, respectively.

In [ ]:
plt.figure(figsize=(12,7))
ax = sns.histplot(data=strokep, x='age', hue='heart_disease', multiple='stack')
plt.title('The distribution of age on people who have got stroke categorized by heart disease histogram');

In [ ]:
plt.figure(figsize=(12,7))
ax = sns.histplot(data=strokep, x='age', hue='hypertension', multiple='stack')
plt.title('The distribution of age on people who have got stroke categorized by hypertension histogram');

We can see from this graph that if the age is higher than 50, more likely to have a stroke. Moreover, many people who have stroke also have hypertension and heart disease.

Lastly, we plot the correlation heatmap to show whether the variables are correlated to each other.

In [ ]:
plt.figure(figsize=(16, 9))
heatmap = sns.heatmap(df.corr(), vmin=-1, vmax=1, annot=True, cmap=sns.color_palette("vlag", as_cmap=True))
heatmap.set_title('Correlation Heatmap', fontdict={'fontsize':14}, pad=12);

As we can see, the most impact variable that affect on the three causes assumption, stroke and bmi is age variable. Below we plot the scatter plot on the bmi and average glucose level variables.

In [ ]:
plt.figure(figsize=(12,7))
ax = sns.scatterplot(data=df, x='bmi', y='avg_glucose_level', marker="x")
plt.title('Scatter plot of bmi and average glucose level');

# Logistic Regression

In [ ]:
from imblearn.under_sampling import RandomUnderSampler
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import sklearn.metrics as metrics

We fist drop irrelevant column (id) before we make the model.

In [ ]:
df_subset = df.drop(['id'], axis = 1).dropna(axis=0).reset_index(drop = True)

Then we use get_dummies to get the numeric data for each categorical variables.

In [ ]:
df_subset = pd.get_dummies(df_subset, drop_first=False)
df_subset

After that, we use under-sample method to sample the imbalance datas. Then we split into train and test data.

In [ ]:
X = df_subset.drop(columns=['stroke'])
y = df_subset['stroke']

undersample = RandomUnderSampler(sampling_strategy=0.6)
X_sampled, y_sampled = undersample.fit_resample(X, y)

X_train, X_test, y_train, y_test = train_test_split(X_sampled, y_sampled,test_size=0.25,random_state=42)

We make pipeline to scaling data first, then we put in the logistic regression model.

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

pipe = make_pipeline(StandardScaler(), LogisticRegression())
pipe.fit(X_train,y_train)

In [ ]:
print('The coefficients are: ', pipe[1].coef_)
print('The y-intercept is: ', pipe[1].intercept_)

Now we make a prediction on the test variable.

In [ ]:
y_pred = pipe.predict(X_test)

These show all the prediction values.

In [ ]:
print(y_pred)

And show the probability of all predictions.

In [ ]:
pipe.predict_proba(X_test)

These are 4 metrics of the prediction. We can see that there is 77% of accuracy, which means we can correctly predict the person whether have got stroke or not by 77 percents.

In [ ]:
table = pd.DataFrame(data = {'Model': "Logistic Regression",
                          'F1': ["%.6f" % metrics.f1_score(y_test, y_pred)],
                          'Recall': ["%.6f" % metrics.recall_score(y_test, y_pred)],
                          'Precision': ["%.6f" % metrics.precision_score(y_test, y_pred)],
                          'Accuracy': ["%.6f" % metrics.accuracy_score(y_test, y_pred)]})
table

And below is the confusion matrix However, we have seen that from people who have stroke, there will be 108 people that we can correctly predict, but the remaining 18 are false negative, which seems to be a large number.

In [ ]:
cm = metrics.confusion_matrix(y_test, y_pred, labels = pipe[1].classes_)
disp = metrics.ConfusionMatrixDisplay(confusion_matrix = cm,display_labels = pipe[1].classes_)
disp.plot()

We plot an ROC curve below, which shows that the prediction are likely to be just a random since AUC is only 0.57. So, we should find more other model to aim more accuracy and AUC.

In [ ]:
from sklearn.metrics import roc_curve, auc, precision_recall_curve, average_precision_score
def ROC_plot(clf,X_test):
  y_score = clf.predict_proba(X_test)
  fpr, tpr, _ = roc_curve(y_test, y_score[:, 1])
  roc_auc = auc(fpr, tpr)

  plt.figure(figsize=(12, 8))

  # ROC
  plt.plot(fpr, tpr, color='black', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
  plt.plot([0, 1], [0, 1], color='red', lw=2, linestyle='--')
  plt.xlim([-0.05, 1.05])
  plt.ylim([-0.05, 1.05])
  plt.xlabel('False Positive Rate')
  plt.ylabel('True Positive Rate')
  plt.title('Receiver operating characteristic (ROC)')
  plt.legend(loc='lower right')
  plt.show()

In [ ]:
ROC_plot(pipe[1],X_test)

# Decision Tree Model

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import plot_tree

We set, fit, and predict the decision tree model.

In [ ]:
decision_tree = DecisionTreeClassifier(random_state=42)

decision_tree.fit(X_train, y_train)

dt_pred = decision_tree.predict(X_test)

Now we have the metric score as below. We can see that most of the metrics are drop from the logistic regression model.

In [ ]:
print("Decision Tree")
print("Accuracy:", "%.6f" % metrics.accuracy_score(y_test, dt_pred))
print("Precision:", "%.6f" % metrics.precision_score(y_test, dt_pred))
print("Recall:", "%.6f" % metrics.recall_score(y_test, dt_pred))
print("F1 Score:", "%.6f" % metrics.f1_score(y_test, dt_pred))

In [ ]:
result_table1 = pd.DataFrame(data = {'Model': "Decision Tree",
                          'F1': ["%.6f" % metrics.f1_score(y_test, dt_pred)],
                          'Recall': ["%.6f" % metrics.recall_score(y_test, dt_pred)],
                          'Precision': ["%.6f" % metrics.precision_score(y_test, dt_pred)],
                          'Accuracy': ["%.6f" % metrics.accuracy_score(y_test, dt_pred)]})
table = pd.concat([table, result_table1], ignore_index=True)

We observe the confusion matrix below. We can see that false negative is quite big.

In [ ]:
cm = metrics.confusion_matrix(y_test, dt_pred, labels = decision_tree.classes_)
disp = metrics.ConfusionMatrixDisplay(confusion_matrix = cm,display_labels = decision_tree.classes_)
disp.plot()

In [ ]:
ROC_plot(decision_tree,X_test)

In [ ]:
plt.figure(figsize=(20,12))
plot_tree(decision_tree, max_depth=2, fontsize=14, feature_names=X.columns);

In [ ]:
importances = decision_tree.feature_importances_

forest_importances = pd.Series(importances, index=X.columns).sort_values(ascending=False)

fig, ax = plt.subplots()
forest_importances.plot.bar(ax=ax);

# Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import PredefinedSplit
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 42)
X_tr, X_val, y_tr, y_val = train_test_split(X_train, y_train, test_size = 0.25, random_state = 42)

We will set the amount of parameters for testing the cross validation.

In [ ]:
cv_params = {'n_estimators' : [50,100], 
              'max_depth' : [10,50],        
              'min_samples_leaf' : [0.5,1], 
              'min_samples_split' : [0.001, 0.01],
              'max_features' : ["sqrt"], 
              'max_samples' : [.5,.9]}

In [ ]:
split_index = [0 if x in X_val.index else -1 for x in X_train.index]
custom_split = PredefinedSplit(split_index)

Then we fit the random forest model.

In [ ]:
rf = RandomForestClassifier(random_state=42, class_weight='balanced')

In [ ]:
rf_val = GridSearchCV(rf, cv_params, cv=custom_split, refit='f1', n_jobs = -1, verbose = 1)

In [ ]:
rf_val.fit(X_train, y_train)

In [ ]:
rf_val.best_params_

In [ ]:
rf_opt = RandomForestClassifier(max_depth = 10,
 max_features = 'sqrt',
 max_samples = 0.5,
 min_samples_leaf = 1,
 min_samples_split = 0.001,
 n_estimators = 50, random_state = 42, class_weight='balanced')

In [ ]:
rf_opt.fit(X_train, y_train)

In [ ]:
y_pred = rf_opt.predict(X_test)

This model have high value of all metrics, which exceed more than 90% for each metric.

In [ ]:
pc_test = precision_score(y_test, y_pred, pos_label = 0)
rc_test = recall_score(y_test, y_pred, pos_label = 0)
ac_test = accuracy_score(y_test, y_pred)
f1_test = f1_score(y_test, y_pred, pos_label = 0)

In [ ]:
table = table.append({'Model': "Tuned Random Forest",
                        'F1':  f1_test,
                        'Recall': rc_test,
                        'Precision': pc_test,
                        'Accuracy': ac_test
                      },
                        ignore_index=True
                    )
table

However, the false negative value are too high than the other model and it can find the true negative only 5 samples

In [ ]:
cm = metrics.confusion_matrix(y_test, y_pred)
disp = metrics.ConfusionMatrixDisplay(confusion_matrix = cm)
disp.plot()

For this model, we have the AUC 0.82, which is fairly high.

In [ ]:
ROC_plot(rf_opt,X_test)

In [ ]:
table